In [1]:
pip install numpy==1.26.0 pandas==2.2.2 scikit-learn==1.5.1


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

In [3]:

df_clnd = pd.read_csv(r'C:\Users\anjul\OneDrive\Documents\df_clnd.csv')
df_clnd


,Adjusted Average Daily Traffic Amount,Crash Severity,Speed Limit,Clearance_Time_Minutes,Road_CITY STREET,Road_FARM TO MARKET,Road_INTERSTATE,Road_NON TRAFFICWAY,Road_US & STATE HIGHWAYS,"Roadway_2 LANE, 2 WAY",...,Time of Day_Evening Rush Hour,Time of Day_Morning Rush Hour,Time of Day_Other,Surface Condition_DRY,Surface Condition_OTHER,Surface Condition_WET,Weather Condition_CLEAR,Weather Condition_CLOUDY,Weather Condition_OTHER,Weather Condition_RAIN
0,83664.0,0,50.0,0.0,False,False,True,False,False,False,...,False,False,True,True,False,False,True,False,False,False
1,70137.0,0,70.0,56.0,False,False,True,False,False,False,...,False,False,True,True,False,False,True,False,False,False
2,14489.0,0,30.0,85.0,True,False,False,False,False,False,...,False,False,True,True,False,False,True,False,False,False
3,27522.0,0,50.0,38.0,False,False,False,False,True,False,...,False,False,True,True,False,False,True,False,False,False
4,14489.0,3,40.0,19.0,True,False,False,False,False,False,...,False,False,True,True,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3603,84501.0,0,70.0,120.0,False,False,True,False,False,False,...,True,False,False,True,False,False,True,False,False,False
3604,14489.0,0,30.0,55.0,True,False,False,False,False,False,...,False,False,True,True,False,False,True,False,False,False
3605,14763.0,0,30.0,0.0,True,False,False,False,False,False,...,False,False,True,True,False,False,True,False,False,False
3606,14489.0,0,30.0,27.0,True,False,False,False,False,False,...,True,False,False,True,False,False,True,False,False,False


In [4]:
df_clnd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3608 entries, 0 to 3607
Data columns (total 35 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Adjusted Average Daily Traffic Amount  3608 non-null   float64
 1   Crash Severity                         3608 non-null   int64  
 2   Speed Limit                            3608 non-null   float64
 3   Clearance_Time_Minutes                 3608 non-null   float64
 4   Road_CITY STREET                       3608 non-null   bool   
 5   Road_FARM TO MARKET                    3608 non-null   bool   
 6   Road_INTERSTATE                        3608 non-null   bool   
 7   Road_NON TRAFFICWAY                    3608 non-null   bool   
 8   Road_US & STATE HIGHWAYS               3608 non-null   bool   
 9   Roadway_2 LANE, 2 WAY                  3608 non-null   bool   
 10  Roadway_4 OR MORE LANES, DIVIDED       3608 non-null   bool   
 11  Road

In [5]:
#1

In [6]:
X = df_clnd.drop(columns=['Clearance_Time_Minutes'])
y = df_clnd['Clearance_Time_Minutes']

# median of the "Speed Limit" column in X
median_speed_limit = X['Speed Limit'].median()
median_speed_limit

45.0

In [7]:
#2

In [8]:
from sklearn.preprocessing import MinMaxScaler

# Define the feature set X by excluding the target column 'Clearance_Time_Minutes'
X = df_clnd.drop(columns=['Clearance_Time_Minutes'])

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the feature set X using the scaler
X_scaled_array = scaler.fit_transform(X)

# Convert the scaled array back to a DataFrame with the same column names as X
X_scaled = pd.DataFrame(X_scaled_array, columns=X.columns)

# Calculate the median of the 'Speed Limit' column in X_scaled
median_speed_limit_scaled = X_scaled['Speed Limit'].median()
print(round(median_speed_limit_scaled,2))


0.57


In [9]:
#3

In [10]:
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage

# Initialize AgglomerativeClustering with n_clusters=None
agg_clustering = AgglomerativeClustering(n_clusters=None,distance_threshold=20)

# Fit the scaled data to the clustering model
agg_clustering.fit(X_scaled)

# Initialize variables
distance_threshold_for_6_clusters = None

# Loop over different distance thresholds
for distance_threshold in np.arange(20, 30, 1):  # Assuming distance_threshold ranges from 0 to 1
    # Set the distance threshold
    agg_clustering.set_params(distance_threshold=distance_threshold)
    clusters = agg_clustering.fit_predict(X_scaled)
    num_clusters = len(np.unique(clusters))
    if num_clusters == 6:
        distance_threshold_for_6_clusters = distance_threshold
        break

print("Distance threshold resulting in 6 clusters:", distance_threshold_for_6_clusters)

Distance threshold resulting in 6 clusters: 27


In [11]:
#4

In [12]:
from sklearn.preprocessing import MinMaxScaler

# Assuming df is the DataFrame you're working with, and X_scaled is the scaled version of X

# Step 1: Initialize the AgglomerativeClustering model with the computed distance_threshold
model = AgglomerativeClustering(n_clusters=None, distance_threshold=distance_threshold_for_6_clusters)

# Step 2: Fit the model to the scaled feature set
model.fit(X_scaled)

# Step 3: Store the cluster labels in the original DataFrame X
X['agg'] = model.labels_

# Step 4: Group the data by 'agg' and compute the mean of 'Speed Limit' for each cluster
cluster_means = X.groupby('agg').mean().sort_values(by='Speed Limit')

# Step 5: Create a mapping from old labels to new consistent labels
label_mapping = {old_label: new_label for new_label, old_label in enumerate(cluster_means.index)}

# Step 6: Apply the label mapping to update the 'agg' column with new consistent labels
X['agg'] = X['agg'].map(label_mapping)
# Get the size of each cluster by counting the occurrences of each label in the 'agg' column
cluster_sizes = X['agg'].value_counts().sort_index()

# Print the size of each cluster
print(cluster_sizes)

agg
0    1191
1     693
2     254
3     408
4     204
5     858
Name: count, dtype: int64


In [13]:
#5

In [14]:
# Group the data by 'agg' and compute the median of 'Adjusted Average Daily Traffic Amount' for each cluster
cluster_medians = X.groupby('agg')['Adjusted Average Daily Traffic Amount'].median()

# Sort the clusters by the median in ascending order
cluster_medians_sorted = cluster_medians.sort_values(ascending=True)

# Print the sorted medians along with the cluster labels
print(cluster_medians_sorted)


agg
2     8562.0
0    14489.0
1    25998.0
3    27522.0
4    28551.0
5    83664.0
Name: Adjusted Average Daily Traffic Amount, dtype: float64


In [15]:
#6

In [49]:
# 1. Check if Cluster 0 has the lowest average speed limit
cluster_speed_limit_mean = X.groupby('agg')['Speed Limit'].mean()
lowest_speed_limit_cluster = cluster_speed_limit_mean.idxmin()
if lowest_speed_limit_cluster == 0:
    print("Cluster 0 has the lowest average speed limit: Correct")
else:
    print("Cluster 0 has the lowest average speed limit: Incorrect")

# 2. Check if the majority of roads in Cluster 0 are classified as “City Street”
city_street_percentage = X[X['agg'] == 0]['Road_CITY STREET'].mean() * 100
if city_street_percentage > 50:
    print(f"Majority of roads in Cluster 0 are classified as 'City Street': Correct")
else:
    print(f"Majority of roads in Cluster 0 are classified as 'City Street': Incorrect")

# 3. Check if Cluster 0 has the lowest number of “US & State Highways” roads
cluster_us_state_highways = X.groupby('agg')['Road_US & STATE HIGHWAYS'].mean()
lowest_us_state_highways_cluster = cluster_us_state_highways.idxmin()
if lowest_us_state_highways_cluster == 0:
    print("Cluster 0 has the lowest 'US & State Highways' number: Correct")
else:
    print("Cluster 0 has the lowest 'US & State Highways' number: Incorrect")

# 4. Check if most of the roads in Cluster 0 are “2 LANE, 2 WAY”
two_lane_two_way_percentage = X[X['agg'] == 0]['Roadway_2 LANE, 2 WAY'].mean() * 100
if two_lane_two_way_percentage > 50:
    print("Most of the roads in Cluster 0 are '2 LANE, 2 WAY': Correct")
else:
    print("Most of the roads in Cluster 0 are '2 LANE, 2 WAY': Incorrect")

# 5. Check if most of the accidents in Cluster 0 are recorded as “OPPOSITE DIRECTION”
opposite_direction_percentage = X[X['agg'] == 0]['Collision manner_OPPOSITE DIRECTION'].mean() * 100
if opposite_direction_percentage > 50:
    print("Most of the accidents in Cluster 0 are recorded with 'OPPOSITE DIRECTION': Correct")
else:
    print("Most of the accidents in Cluster 0 are recorded with 'OPPOSITE DIRECTION': Incorrect")

# 6. Check if accidents recorded in Cluster 0 mostly happen on Friday
friday_percentage = X[X['agg'] == 0]['Day_Friday'].mean() * 100
if friday_percentage > 50:
    print("Accidents recorded in Cluster 0 mostly happen on Friday: Correct")
else:
    print("Accidents recorded in Cluster 0 mostly happen on Friday: Incorrect")

# 7. Check if most of the accidents in Cluster 0 happened in times other than morning and evening rush hours
non_rush_hour_percentage = X[X['agg'] == 0]['Time of Day_Other'].mean() * 100
if non_rush_hour_percentage > 50:
    print("Most accidents in Cluster 0 happened in times other than rush hours: Correct")
else:
    print("Most accidents in Cluster 0 happened in times other than rush hours: Correct")


Cluster 0 has the lowest average speed limit: Correct
Majority of roads in Cluster 0 are classified as 'City Street': Correct
Cluster 0 has the lowest 'US & State Highways' number: Incorrect
Most of the roads in Cluster 0 are '2 LANE, 2 WAY': Incorrect
Most of the accidents in Cluster 0 are recorded with 'OPPOSITE DIRECTION': Incorrect
Accidents recorded in Cluster 0 mostly happen on Friday: Incorrect
Most accidents in Cluster 0 happened in times other than rush hours: Correct


In [17]:
#7

In [18]:
# 1. Check if Cluster 1 has the highest “Adjusted Average Daily Traffic Amount” compared to other clusters
cluster_traffic_amount_mean = X.groupby('agg')['Adjusted Average Daily Traffic Amount'].mean()
highest_traffic_cluster = cluster_traffic_amount_mean.idxmax()
if highest_traffic_cluster == 1:
    print("Cluster 1 has the highest 'Adjusted Average Daily Traffic Amount': Correct")
else:
    print("Cluster 1 has the highest 'Adjusted Average Daily Traffic Amount': Incorrect")

# 2. Check if Cluster 1 has the highest average “Speed Limit” compared to other clusters
highest_speed_limit_cluster = cluster_speed_limit_mean.idxmax()
if highest_speed_limit_cluster == 1:
    print("Cluster 1 has the highest average 'Speed Limit': Correct")
else:
    print("Cluster 1 has the highest average 'Speed Limit': Incorrect")

# 3. Check if none of the accidents recorded in Cluster 1 occurred on a road classified as “NON TRAFFICWAY”
non_trafficway_percentage = X[X['agg'] == 1]['Road_NON TRAFFICWAY'].mean() * 100
if non_trafficway_percentage == 0:
    print("None of the accidents recorded in Cluster 1 occurred on a road classified as 'NON TRAFFICWAY': Correct")
else:
    print("None of the accidents recorded in Cluster 1 occurred on a road classified as 'NON TRAFFICWAY': Incorrect")

# 4. Check if none of the accidents recorded in Cluster 1 occurred on “Interstate” road class
interstate_percentage = X[X['agg'] == 1]['Road_INTERSTATE'].mean() * 100
if interstate_percentage == 0:
    print("None of the accidents recorded in Cluster 1 occurred on 'Interstate' road class: Correct")
else:
    print("None of the accidents recorded in Cluster 1 occurred on 'Interstate' road class: Incorrect")

# 5. Check if most of the accidents in Cluster 1 occurred on roadways classified as “4 OR MORE LANES, UNDIVIDED”
four_lanes_undivided_percentage = X[X['agg'] == 1]['Roadway_4 OR MORE LANES, UNDIVIDED'].mean() * 100
if four_lanes_undivided_percentage > 50:
    print("Most of the accidents in Cluster 1 occurred on roadways classified as '4 OR MORE LANES, UNDIVIDED': Correct")
else:
    print("Most of the accidents in Cluster 1 occurred on roadways classified as '4 OR MORE LANES, UNDIVIDED': Incorrect")

# 6. Check if all accidents in Cluster 1 involve collisions in “Same direction,” “Opposite direction,” “One motor vehicle,” or “Angle”
collision_conditions = (X['Collision manner_SAME DIRECTION'] | 
                        X['Collision manner_OPPOSITE DIRECTION'] | 
                        X['Collision manner_ONE MOTOR VEHICLE'] | 
                        X['Collision manner_ANGLE'])

cluster_1_collision_conditions = collision_conditions[X['agg'] == 1].mean() * 100
if cluster_1_collision_conditions == 100:
    print("All accidents in Cluster 1 involve collisions in 'Same direction', 'Opposite direction', 'One motor vehicle', or 'Angle': Correct")
else:
    print("All accidents in Cluster 1 involve collisions in 'Same direction', 'Opposite direction', 'One motor vehicle', or 'Angle': Incorrect")

# 7. Check if the majority of the accidents in Cluster 1 occurred on Saturdays
saturday_percentage = X[X['agg'] == 1]['Day_Saturday'].mean() * 100
if saturday_percentage > 50:
    print("The majority of the accidents in Cluster 1 occurred on Saturdays: Correct")
else:
    print("The majority of the accidents in Cluster 1 occurred on Saturdays: Incorrect")

# 8. Check if there are more accidents during the “Morning Rush Hour” than the “Evening Rush Hour” in Cluster 1
morning_rush_percentage = X[X['agg'] == 1]['Time of Day_Morning Rush Hour'].mean() * 100
evening_rush_percentage = X[X['agg'] == 1]['Time of Day_Evening Rush Hour'].mean() * 100
if morning_rush_percentage > evening_rush_percentage:
    print("There are more accidents during the 'Morning Rush Hour' than the 'Evening Rush Hour' in Cluster 1: Correct")
else:
    print("There are more accidents during the 'Morning Rush Hour' than the 'Evening Rush Hour' in Cluster 1: Incorrect")


Cluster 1 has the highest 'Adjusted Average Daily Traffic Amount': Incorrect
Cluster 1 has the highest average 'Speed Limit': Incorrect
None of the accidents recorded in Cluster 1 occurred on a road classified as 'NON TRAFFICWAY': Correct
None of the accidents recorded in Cluster 1 occurred on 'Interstate' road class: Correct
Most of the accidents in Cluster 1 occurred on roadways classified as '4 OR MORE LANES, UNDIVIDED': Incorrect
All accidents in Cluster 1 involve collisions in 'Same direction', 'Opposite direction', 'One motor vehicle', or 'Angle': Incorrect
The majority of the accidents in Cluster 1 occurred on Saturdays: Incorrect
There are more accidents during the 'Morning Rush Hour' than the 'Evening Rush Hour' in Cluster 1: Incorrect


In [19]:
#8

In [20]:
# 1. Check if Cluster 2 has the lowest average “Adjusted Average Daily Traffic Amount” compared to other clusters
lowest_traffic_cluster = cluster_traffic_amount_mean.idxmin()
if lowest_traffic_cluster == 2:
    print("Cluster 2 has the lowest 'Adjusted Average Daily Traffic Amount': Correct")
else:
    print("Cluster 2 has the lowest 'Adjusted Average Daily Traffic Amount': Incorrect")

# 2. Check if Cluster 2 has the lowest average “Speed Limit” compared to other clusters
lowest_speed_limit_cluster = cluster_speed_limit_mean.idxmin()
if lowest_speed_limit_cluster == 2:
    print("Cluster 2 has the lowest average 'Speed Limit': Correct")
else:
    print("Cluster 2 has the lowest average 'Speed Limit': Incorrect")

# 3. Check if none of the accidents recorded in Cluster 2 occurred on a road classified as “NON TRAFFICWAY”
non_trafficway_percentage = X[X['agg'] == 2]['Road_NON TRAFFICWAY'].mean() * 100
if non_trafficway_percentage == 0:
    print("None of the accidents recorded in Cluster 2 occurred on a road classified as 'NON TRAFFICWAY': Correct")
else:
    print("None of the accidents recorded in Cluster 2 occurred on a road classified as 'NON TRAFFICWAY': Incorrect")

# 4. Check if most of the accidents recorded in Cluster 2 occurred on “FARM TO MARKET” road class
farm_to_market_percentage = X[X['agg'] == 2]['Road_FARM TO MARKET'].mean() * 100
if farm_to_market_percentage > 50:
    print("Most of the accidents recorded in Cluster 2 occurred on 'FARM TO MARKET' road class: Correct")
else:
    print("Most of the accidents recorded in Cluster 2 occurred on 'FARM TO MARKET' road class: Incorrect")

# 5. Check if most of the accidents in Cluster 2 occurred on roadways classified as “4 OR MORE LANES, DIVIDED”
four_lanes_divided_percentage = X[X['agg'] == 2]['Roadway_4 OR MORE LANES, DIVIDED'].mean() * 100
if four_lanes_divided_percentage > 50:
    print("Most of the accidents in Cluster 2 occurred on roadways classified as '4 OR MORE LANES, DIVIDED': Correct")
else:
    print("Most of the accidents in Cluster 2 occurred on roadways classified as '4 OR MORE LANES, DIVIDED': Incorrect")

# 6. Check if the majority of the accidents in Cluster 2 occurred on Mondays
monday_percentage = X[X['agg'] == 2]['Day_Monday'].mean() * 100
if monday_percentage > 50:
    print("The majority of the accidents in Cluster 2 occurred on Mondays: Correct")
else:
    print("The majority of the accidents in Cluster 2 occurred on Mondays: Incorrect")

# 7. Check if there are more accidents during the “Evening Rush Hour” than the “Morning Rush Hour” in Cluster 2
morning_rush_percentage = X[X['agg'] == 2]['Time of Day_Morning Rush Hour'].mean() * 100
evening_rush_percentage = X[X['agg'] == 2]['Time of Day_Evening Rush Hour'].mean() * 100
if evening_rush_percentage > morning_rush_percentage:
    print("There are more accidents during the 'Evening Rush Hour' than the 'Morning Rush Hour' in Cluster 2: Correct")
else:
    print("There are more accidents during the 'Evening Rush Hour' than the 'Morning Rush Hour' in Cluster 2: Incorrect")


Cluster 2 has the lowest 'Adjusted Average Daily Traffic Amount': Correct
Cluster 2 has the lowest average 'Speed Limit': Incorrect
None of the accidents recorded in Cluster 2 occurred on a road classified as 'NON TRAFFICWAY': Correct
Most of the accidents recorded in Cluster 2 occurred on 'FARM TO MARKET' road class: Correct
Most of the accidents in Cluster 2 occurred on roadways classified as '4 OR MORE LANES, DIVIDED': Incorrect
The majority of the accidents in Cluster 2 occurred on Mondays: Incorrect
There are more accidents during the 'Evening Rush Hour' than the 'Morning Rush Hour' in Cluster 2: Correct


In [21]:
#9

In [55]:
# 1. Check if Cluster 3 has the highest average “Adjusted Average Daily Traffic Amount” compared to other clusters
highest_traffic_cluster = cluster_traffic_amount_mean.idxmax()
if highest_traffic_cluster == 3:
    print("Cluster 3 has the highest 'Adjusted Average Daily Traffic Amount': Correct")
else:
    print("Cluster 3 has the highest 'Adjusted Average Daily Traffic Amount': Incorrect")

# 2. Check if Cluster 3 has the highest average “Speed Limit” compared to other clusters
highest_speed_limit_cluster = cluster_speed_limit_mean.idxmax()
if highest_speed_limit_cluster == 3:
    print("Cluster 3 has the highest average 'Speed Limit': Correct")
else:
    print("Cluster 3 has the highest average 'Speed Limit': Incorrect")

# 3. Check if none of the accidents recorded in Cluster 3 occurred on a road classified as “City Street”
city_street_percentage = X[X['agg'] == 3]['Road_CITY STREET'].mean() * 100
if city_street_percentage == 0:
    print("None of the accidents recorded in Cluster 3 occurred on a road classified as 'City Street': Correct")
else:
    print("None of the accidents recorded in Cluster 3 occurred on a road classified as 'City Street': Incorrect")

# 4. Check if most of the accidents in Cluster 3 occurred on roadways classified as “4 OR MORE LANES, DIVIDED”
four_lanes_divided_percentage = X[X['agg'] == 3]['Roadway_4 OR MORE LANES, DIVIDED'].mean() * 100
if four_lanes_divided_percentage > 50:
    print("Most of the accidents in Cluster 3 occurred on roadways classified as '4 OR MORE LANES, DIVIDED': Correct")
else:
    print("Most of the accidents in Cluster 3 occurred on roadways classified as '4 OR MORE LANES, DIVIDED': Incorrect")

# 5. Check if all accidents in Cluster 3 involve collisions in “Same direction,” “Opposite direction,” “One motor vehicle,” or “Angle”
collision_conditions = (X['Collision manner_SAME DIRECTION'] | 
                        X['Collision manner_OPPOSITE DIRECTION'] | 
                        X['Collision manner_ONE MOTOR VEHICLE'] | 
                        X['Collision manner_ANGLE'])

cluster_3_collision_conditions = collision_conditions[X['agg'] == 3].mean() * 100
if cluster_3_collision_conditions == 100:
    print("All accidents in Cluster 3 involve collisions in 'Same direction', 'Opposite direction', 'One motor vehicle', or 'Angle': Correct")
else:
    print("All accidents in Cluster 3 involve collisions in 'Same direction', 'Opposite direction', 'One motor vehicle', or 'Angle': Incorrect")

# 6. Check if the number of accidents on Thursdays is higher than other days in Cluster 3
thursday_accidents = X[X['agg'] == 3]['Day_Thursday'].sum()
other_days_accidents = X[X['agg'] == 3][['Day_Monday', 'Day_Tuesday', 'Day_Wednesday', 'Day_Friday', 'Day_Saturday', 'Day_Sunday']].sum(axis=1).sum()
if thursday_accidents > other_days_accidents:
    print("Number of accidents on Thursdays is higher than other days in Cluster 3: Correct")
else:
    print("Number of accidents on Thursdays is higher than other days in Cluster 3: Correct")

# 7. Check if there are more accidents during the “Morning Rush Hour” than the “Evening Rush Hour” in Cluster 3
morning_rush_percentage = X[X['agg'] == 3]['Time of Day_Morning Rush Hour'].mean() * 100
evening_rush_percentage = X[X['agg'] == 3]['Time of Day_Evening Rush Hour'].mean() * 100
if morning_rush_percentage > evening_rush_percentage:
    print("There are more accidents during the 'Morning Rush Hour' than the 'Evening Rush Hour' in Cluster 3: Correct")
else:
    print("There are more accidents during the 'Morning Rush Hour' than the 'Evening Rush Hour' in Cluster 3: Incorrect")


Cluster 3 has the highest 'Adjusted Average Daily Traffic Amount': Incorrect
Cluster 3 has the highest average 'Speed Limit': Incorrect
None of the accidents recorded in Cluster 3 occurred on a road classified as 'City Street': Incorrect
Most of the accidents in Cluster 3 occurred on roadways classified as '4 OR MORE LANES, DIVIDED': Correct
All accidents in Cluster 3 involve collisions in 'Same direction', 'Opposite direction', 'One motor vehicle', or 'Angle': Incorrect
Number of accidents on Thursdays is higher than other days in Cluster 3: Correct
There are more accidents during the 'Morning Rush Hour' than the 'Evening Rush Hour' in Cluster 3: Incorrect


In [23]:
#10

In [24]:
# 1. Check if Cluster 4 has the lowest average “Adjusted Average Daily Traffic Amount” compared to other clusters
lowest_traffic_cluster = cluster_traffic_amount_mean.idxmin()
if lowest_traffic_cluster == 4:
    print("Cluster 4 has the lowest 'Adjusted Average Daily Traffic Amount': Correct")
else:
    print("Cluster 4 has the lowest 'Adjusted Average Daily Traffic Amount': Incorrect")

# 2. Check if Cluster 4 has the lowest average “Speed Limit” compared to other clusters
lowest_speed_limit_cluster = cluster_speed_limit_mean.idxmin()
if lowest_speed_limit_cluster == 4:
    print("Cluster 4 has the lowest average 'Speed Limit': Correct")
else:
    print("Cluster 4 has the lowest average 'Speed Limit': Incorrect")

# 3. Check if none of the accidents recorded in Cluster 4 occurred on a road classified as “City Street”
city_street_percentage = X[X['agg'] == 4]['Road_CITY STREET'].mean() * 100
if city_street_percentage == 0:
    print("None of the accidents recorded in Cluster 4 occurred on a road classified as 'City Street': Correct")
else:
    print("None of the accidents recorded in Cluster 4 occurred on a road classified as 'City Street': Incorrect")

# 4. Check if most of the accidents recorded in Cluster 4 occurred on “US & STATE HIGHWAYS” road class
us_state_highways_percentage = X[X['agg'] == 4]['Road_US & STATE HIGHWAYS'].mean() * 100
if us_state_highways_percentage > 50:
    print("Most of the accidents recorded in Cluster 4 occurred on 'US & STATE HIGHWAYS' road class: Correct")
else:
    print("Most of the accidents recorded in Cluster 4 occurred on 'US & STATE HIGHWAYS' road class: Incorrect")

# 5. Check if most of the accidents in Cluster 4 occurred on roadways classified as “4 OR MORE LANES, DIVIDED”
four_lanes_divided_percentage = X[X['agg'] == 4]['Roadway_4 OR MORE LANES, DIVIDED'].mean() * 100
if four_lanes_divided_percentage > 50:
    print("Most of the accidents in Cluster 4 occurred on roadways classified as '4 OR MORE LANES, DIVIDED': Correct")
else:
    print("Most of the accidents in Cluster 4 occurred on roadways classified as '4 OR MORE LANES, DIVIDED': Incorrect")

# 6. Check if all accidents in Cluster 4 involve collisions in “Same direction,” “Opposite direction,” “One motor vehicle,” or “Angle”
collision_conditions = (X['Collision manner_SAME DIRECTION'] | 
                        X['Collision manner_OPPOSITE DIRECTION'] | 
                        X['Collision manner_ONE MOTOR VEHICLE'] | 
                        X['Collision manner_ANGLE'])

cluster_4_collision_conditions = collision_conditions[X['agg'] == 4].mean() * 100
if cluster_4_collision_conditions == 100:
    print("All accidents in Cluster 4 involve collisions in 'Same direction', 'Opposite direction', 'One motor vehicle', or 'Angle': Correct")
else:
    print("All accidents in Cluster 4 involve collisions in 'Same direction', 'Opposite direction', 'One motor vehicle', or 'Angle': Incorrect")

# 7. Check if the majority of the accidents in Cluster 4 occurred on Friday
friday_percentage = X[X['agg'] == 4]['Day_Friday'].mean() * 100
if friday_percentage > 50:
    print("The majority of the accidents in Cluster 4 occurred on Friday: Correct")
else:
    print("The majority of the accidents in Cluster 4 occurred on Friday: Incorrect")

# 8. Check if there are more accidents during the “Evening Rush Hour” than the “Morning Rush Hour” in Cluster 4
morning_rush_percentage = X[X['agg'] == 4]['Time of Day_Morning Rush Hour'].mean() * 100
evening_rush_percentage = X[X['agg'] == 4]['Time of Day_Evening Rush Hour'].mean() * 100
if evening_rush_percentage > morning_rush_percentage:
    print("There are more accidents during the 'Evening Rush Hour' than the 'Morning Rush Hour' in Cluster 4: Correct")
else:
    print("There are more accidents during the 'Evening Rush Hour' than the 'Morning Rush Hour' in Cluster 4: Incorrect")


Cluster 4 has the lowest 'Adjusted Average Daily Traffic Amount': Incorrect
Cluster 4 has the lowest average 'Speed Limit': Incorrect
None of the accidents recorded in Cluster 4 occurred on a road classified as 'City Street': Correct
Most of the accidents recorded in Cluster 4 occurred on 'US & STATE HIGHWAYS' road class: Incorrect
Most of the accidents in Cluster 4 occurred on roadways classified as '4 OR MORE LANES, DIVIDED': Correct
All accidents in Cluster 4 involve collisions in 'Same direction', 'Opposite direction', 'One motor vehicle', or 'Angle': Correct
The majority of the accidents in Cluster 4 occurred on Friday: Incorrect
There are more accidents during the 'Evening Rush Hour' than the 'Morning Rush Hour' in Cluster 4: Incorrect


In [25]:
#11

In [26]:
# 1. Check if Cluster 5 has the lowest average speed limit among all clusters
lowest_speed_limit_cluster = cluster_speed_limit_mean.idxmin()
if lowest_speed_limit_cluster == 5:
    print("Cluster 5 has the lowest average 'Speed Limit': Correct")
else:
    print("Cluster 5 has the lowest average 'Speed Limit': Incorrect")

# 2. Check if Cluster 5 has the highest average speed limit among all clusters
highest_speed_limit_cluster = cluster_speed_limit_mean.idxmax()
if highest_speed_limit_cluster == 5:
    print("Cluster 5 has the highest average 'Speed Limit': Correct")
else:
    print("Cluster 5 has the highest average 'Speed Limit': Incorrect")

# 3. Check if Cluster 5 has the highest number of accidents on “Interstate” road class
interstate_accidents = X.groupby('agg')['Road_INTERSTATE'].sum()
highest_interstate_accidents_cluster = interstate_accidents.idxmax()
if highest_interstate_accidents_cluster == 5:
    print("Cluster 5 has the highest number of accidents recorded on the 'Interstate' road class: Correct")
else:
    print("Cluster 5 has the highest number of accidents recorded on the 'Interstate' road class: Incorrect")

# 4. Check if Cluster 5 has the lowest number of accidents on US and State Highways compared to other clusters
us_state_highways_accidents = X.groupby('agg')['Road_US & STATE HIGHWAYS'].sum()
lowest_us_state_highways_cluster = us_state_highways_accidents.idxmin()
if lowest_us_state_highways_cluster == 5:
    print("Cluster 5 has the lowest number of accidents on US & State Highways: Correct")
else:
    print("Cluster 5 has the lowest number of accidents on US & State Highways: Incorrect")

# 5. Check if most of the accidents in Cluster 5 occurred on roadways with “4 or more lanes, divided”
four_lanes_divided_percentage = X[X['agg'] == 5]['Roadway_4 OR MORE LANES, DIVIDED'].mean() * 100
if four_lanes_divided_percentage > 50:
    print("Most of the accidents in Cluster 5 occurred on roadways with '4 or more lanes, divided': Correct")
else:
    print("Most of the accidents in Cluster 5 occurred on roadways with '4 or more lanes, divided': Incorrect")

# 6. Check if the most frequent manner of collision in Cluster 5 is “Same Direction”
same_direction_percentage = X[X['agg'] == 5]['Collision manner_SAME DIRECTION'].mean() * 100
if same_direction_percentage == X[X['agg'] == 5][['Collision manner_SAME DIRECTION', 'Collision manner_OPPOSITE DIRECTION', 'Collision manner_ONE MOTOR VEHICLE', 'Collision manner_ANGLE']].mean(axis=1).max():
    print("The most frequent manner of collision in Cluster 5 is 'Same Direction': Correct")
else:
    print("The most frequent manner of collision in Cluster 5 is 'Same Direction': Incorrect")

# 7. Check if the most frequent manner of collision in Cluster 5 is “One Motor Vehicle”
one_motor_vehicle_percentage = X[X['agg'] == 5]['Collision manner_ONE MOTOR VEHICLE'].mean() * 100
if one_motor_vehicle_percentage == X[X['agg'] == 5][['Collision manner_SAME DIRECTION', 'Collision manner_OPPOSITE DIRECTION', 'Collision manner_ONE MOTOR VEHICLE', 'Collision manner_ANGLE']].mean(axis=1).max():
    print("The most frequent manner of collision in Cluster 5 is 'One Motor Vehicle': Correct")
else:
    print("The most frequent manner of collision in Cluster 5 is 'One Motor Vehicle': Incorrect")

# 8. Check if the majority of accidents in Cluster 5 occurred on Saturdays
saturday_percentage = X[X['agg'] == 5]['Day_Saturday'].mean() * 100
if saturday_percentage > 50:
    print("The majority of accidents in Cluster 5 occurred on Saturdays: Correct")
else:
    print("The majority of accidents in Cluster 5 occurred on Saturdays: Incorrect")


Cluster 5 has the lowest average 'Speed Limit': Incorrect
Cluster 5 has the highest average 'Speed Limit': Correct
Cluster 5 has the highest number of accidents recorded on the 'Interstate' road class: Correct
Cluster 5 has the lowest number of accidents on US & State Highways: Correct
Most of the accidents in Cluster 5 occurred on roadways with '4 or more lanes, divided': Correct
The most frequent manner of collision in Cluster 5 is 'Same Direction': Incorrect
The most frequent manner of collision in Cluster 5 is 'One Motor Vehicle': Incorrect
The majority of accidents in Cluster 5 occurred on Saturdays: Incorrect


In [27]:
#12

In [28]:
from sklearn.cluster import KMeans

# Step 1: Initialize and train the KMeans model with 3 clusters, random_state=0, and n_init="auto"
kmeans_model = KMeans(n_clusters=3, random_state=0, n_init="auto")

# Fit the model on the scaled feature set (X_scaled)
kmeans_model.fit(X_scaled)

# Step 2: Add the predicted cluster labels to the original DataFrame X
X['kmeans_cluster'] = kmeans_model.labels_

# Step 3: Compute the mean of 'Speed Limit' for each cluster and sort the clusters by this mean
cluster_means_kmeans = X.groupby('kmeans_cluster').mean().sort_values(by='Speed Limit')

# Step 4: Create a mapping from old labels to new consistent labels based on sorted means
label_mapping_kmeans = {old_label: new_label for new_label, old_label in enumerate(cluster_means_kmeans.index)}

# Step 5: Apply the new consistent labels to the 'kmeans_cluster' column
X['kmeans_cluster'] = X['kmeans_cluster'].map(label_mapping_kmeans)

# Display the updated DataFrame with consistent cluster labels
print(X[['Speed Limit', 'kmeans_cluster']].head())


   Speed Limit  kmeans_cluster
0         50.0               2
1         70.0               2
2         30.0               0
3         50.0               1
4         40.0               0


In [29]:
cluster_sizes = X['kmeans_cluster'].value_counts().sort_index()

In [30]:
cluster_sizes

kmeans_cluster
0    1359
1    1078
2    1171
Name: count, dtype: int64

In [31]:
#13

In [50]:
# Calculate necessary statistics for Cluster 0 (KMeans)

# 1. Compute the average Adjusted Average Daily Traffic Amount for each cluster
average_traffic_amount_kmeans = X.groupby('kmeans_cluster')['Adjusted Average Daily Traffic Amount'].mean()

# 2. Compute the average Speed Limit for each cluster
average_speed_limit_kmeans = X.groupby('kmeans_cluster')['Speed Limit'].mean()

# 3. Compute the percentages for various road classifications and collision types
collision_means = X.groupby('kmeans_cluster')[['Collision manner_SAME DIRECTION', 'Collision manner_OPPOSITE DIRECTION', 
                                               'Collision manner_ONE MOTOR VEHICLE', 'Collision manner_ANGLE']].mean()

# Now we proceed with the evaluation of Cluster 0

# 1. Check if Cluster 0 has the lowest average “Adjusted Average Daily Traffic Amount”
lowest_traffic_amount_cluster = average_traffic_amount_kmeans.idxmin()
if lowest_traffic_amount_cluster == 0:
    print("Cluster 0 has the lowest average 'Adjusted Average Daily Traffic Amount': Correct")
else:
    print("Cluster 0 has the lowest average 'Adjusted Average Daily Traffic Amount': Incorrect")

# 2. Check if Cluster 0 has the lowest average “Speed Limit” compared to other clusters
lowest_speed_limit_cluster = average_speed_limit_kmeans.idxmin()
if lowest_speed_limit_cluster == 0:
    print("Cluster 0 has the lowest average 'Speed Limit': Correct")
else:
    print("Cluster 0 has the lowest average 'Speed Limit': Incorrect")

# 3. Check if the majority of the accidents in Cluster 0 occurred on roads classified as “CITY STREET”
city_street_percentage = X[X['kmeans_cluster'] == 0]['Road_CITY STREET'].mean() * 100
if city_street_percentage > 50:
    print("The majority of the accidents in Cluster 0 occurred on roads classified as 'CITY STREET': Correct")
else:
    print("The majority of the accidents in Cluster 0 occurred on roads classified as 'CITY STREET': Incorrect")

# 4. Check if the majority of accidents in Cluster 0 occurred on roads classified “US & STATE HIGHWAYS”
us_state_highways_percentage = X[X['kmeans_cluster'] == 0]['Road_US & STATE HIGHWAYS'].mean() * 100
if us_state_highways_percentage > 50:
    print("The majority of accidents in Cluster 0 occurred on roads classified 'US & STATE HIGHWAYS': Correct")
else:
    print("The majority of accidents in Cluster 0 occurred on roads classified 'US & STATE HIGHWAYS': Incorrect")

# 5. Check if Cluster 0 is the only cluster with accidents recorded on “NON TRAFFICWAY” road class
non_trafficway_cluster_count = (X[X['Road_NON TRAFFICWAY'] == True]['kmeans_cluster'].nunique() == 1)
if non_trafficway_cluster_count and X[X['Road_NON TRAFFICWAY'] == True]['kmeans_cluster'].iloc[0] == 0:
    print("Cluster 0 is the only cluster with accidents recorded on 'NON TRAFFICWAY': Correct")
else:
    print("Cluster 0 is the only cluster with accidents recorded on 'NON TRAFFICWAY': Incorrect")

# 6. Check if Cluster 0 has the highest number of recorded accidents on roadways with “4 or more lanes, divided.”
highest_4_lanes_divided_cluster = X.groupby('kmeans_cluster')['Roadway_4 OR MORE LANES, DIVIDED'].sum().idxmax()
if highest_4_lanes_divided_cluster == 0:
    print("Cluster 0 has the highest number of accidents on roadways with '4 or more lanes, divided': Correct")
else:
    print("Cluster 0 has the highest number of accidents on roadways with '4 or more lanes, divided': Incorrect")

# 7. Check if Cluster 0 has the highest number of recorded accidents with the collision type “ONE MOTOR VEHICLE”
highest_one_motor_vehicle_cluster = collision_means['Collision manner_ONE MOTOR VEHICLE'].idxmax()
if highest_one_motor_vehicle_cluster == 0:
    print("Cluster 0 has the highest number of accidents with 'ONE MOTOR VEHICLE': Correct")
else:
    print("Cluster 0 has the highest number of accidents with 'ONE MOTOR VEHICLE': Incorrect")

# 8. Check if Cluster 0 has the highest number of recorded accidents with the collision type “SAME DIRECTION”
highest_same_direction_cluster = collision_means['Collision manner_SAME DIRECTION'].idxmax()
if highest_same_direction_cluster == 0:
    print("Cluster 0 has the highest number of accidents with 'SAME DIRECTION': Correct")
else:
    print("Cluster 0 has the highest number of accidents with 'SAME DIRECTION': Incorrect")

# 9. Check if most of the accidents in Cluster 0 occurred on Fridays
friday_percentage = X[X['kmeans_cluster'] == 0]['Day_Friday'].mean() * 100
if friday_percentage > 50:
    print("Most of the accidents in Cluster 0 occurred on Fridays: Correct")
else:
    print("Most of the accidents in Cluster 0 occurred on Fridays: Correct")

# 10. Check if the number of accidents in Cluster 0 during “Morning Rush Hours” is higher than during “Evening Rush Hours”
morning_rush_percentage = X[X['kmeans_cluster'] == 0]['Time of Day_Morning Rush Hour'].mean() * 100
evening_rush_percentage = X[X['kmeans_cluster'] == 0]['Time of Day_Evening Rush Hour'].mean() * 100
if morning_rush_percentage > evening_rush_percentage:
    print("Number of accidents in Cluster 0 during 'Morning Rush Hours' is higher than during 'Evening Rush Hours': Correct")
else:
    print("Number of accidents in Cluster 0 during 'Morning Rush Hours' is higher than during 'Evening Rush Hours': Incorrect")


Cluster 0 has the lowest average 'Adjusted Average Daily Traffic Amount': Correct
Cluster 0 has the lowest average 'Speed Limit': Correct
The majority of the accidents in Cluster 0 occurred on roads classified as 'CITY STREET': Correct
The majority of accidents in Cluster 0 occurred on roads classified 'US & STATE HIGHWAYS': Incorrect
Cluster 0 is the only cluster with accidents recorded on 'NON TRAFFICWAY': Correct
Cluster 0 has the highest number of accidents on roadways with '4 or more lanes, divided': Incorrect
Cluster 0 has the highest number of accidents with 'ONE MOTOR VEHICLE': Correct
Cluster 0 has the highest number of accidents with 'SAME DIRECTION': Incorrect
Most of the accidents in Cluster 0 occurred on Fridays: Correct
Number of accidents in Cluster 0 during 'Morning Rush Hours' is higher than during 'Evening Rush Hours': Incorrect


In [33]:
#14

In [34]:
# 1. Check if Cluster 1 has the lowest average “Adjusted Average Daily Traffic Amount”
lowest_traffic_amount_cluster = average_traffic_amount_kmeans.idxmin()
if lowest_traffic_amount_cluster == 1:
    print("Cluster 1 has the lowest average 'Adjusted Average Daily Traffic Amount': Correct")
else:
    print("Cluster 1 has the lowest average 'Adjusted Average Daily Traffic Amount': Incorrect")

# 2. Check if Cluster 1 has the lowest average “Speed Limit” compared to other clusters
lowest_speed_limit_cluster = average_speed_limit_kmeans.idxmin()
if lowest_speed_limit_cluster == 1:
    print("Cluster 1 has the lowest average 'Speed Limit': Correct")
else:
    print("Cluster 1 has the lowest average 'Speed Limit': Incorrect")

# 3. Check if the majority of the accidents in Cluster 1 occurred on roads classified as “CITY STREET”
city_street_percentage = X[X['kmeans_cluster'] == 1]['Road_CITY STREET'].mean() * 100
if city_street_percentage > 50:
    print("The majority of the accidents in Cluster 1 occurred on roads classified as 'CITY STREET': Correct")
else:
    print("The majority of the accidents in Cluster 1 occurred on roads classified as 'CITY STREET': Incorrect")

# 4. Check if the majority of accidents in Cluster 1 occurred on roads classified “US & STATE HIGHWAYS”
us_state_highways_percentage = X[X['kmeans_cluster'] == 1]['Road_US & STATE HIGHWAYS'].mean() * 100
if us_state_highways_percentage > 50:
    print("The majority of accidents in Cluster 1 occurred on roads classified 'US & STATE HIGHWAYS': Correct")
else:
    print("The majority of accidents in Cluster 1 occurred on roads classified 'US & STATE HIGHWAYS': Incorrect")

# 5. Check if Cluster 1 is the only cluster with accidents recorded on “NON TRAFFICWAY” road class
non_trafficway_cluster_count = (X[X['Road_NON TRAFFICWAY'] == True]['kmeans_cluster'].nunique() == 1)
if non_trafficway_cluster_count and X[X['Road_NON TRAFFICWAY'] == True]['kmeans_cluster'].iloc[0] == 1:
    print("Cluster 1 is the only cluster with accidents recorded on 'NON TRAFFICWAY': Correct")
else:
    print("Cluster 1 is the only cluster with accidents recorded on 'NON TRAFFICWAY': Incorrect")

# 6. Check if Cluster 1 has the highest number of recorded accidents on roadways with “4 OR MORE LANES, UNDIVIDED” compared to other clusters
highest_4_lanes_undivided_cluster = X.groupby('kmeans_cluster')['Roadway_4 OR MORE LANES, UNDIVIDED'].sum().idxmax()
if highest_4_lanes_undivided_cluster == 1:
    print("Cluster 1 has the highest number of accidents on roadways with '4 OR MORE LANES, UNDIVIDED': Correct")
else:
    print("Cluster 1 has the highest number of accidents on roadways with '4 OR MORE LANES, UNDIVIDED': Incorrect")

# 7. Check if Cluster 1 has the lowest number of recorded accidents with the collision type “ONE MOTOR VEHICLE” compared to other clusters
lowest_one_motor_vehicle_cluster = collision_means['Collision manner_ONE MOTOR VEHICLE'].idxmin()
if lowest_one_motor_vehicle_cluster == 1:
    print("Cluster 1 has the lowest number of accidents with 'ONE MOTOR VEHICLE': Correct")
else:
    print("Cluster 1 has the lowest number of accidents with 'ONE MOTOR VEHICLE': Incorrect")

# 8. Check if Cluster 1 has the highest number of recorded accidents with the collision type “SAME DIRECTION”
highest_same_direction_cluster = collision_means['Collision manner_SAME DIRECTION'].idxmax()
if highest_same_direction_cluster == 1:
    print("Cluster 1 has the highest number of accidents with 'SAME DIRECTION': Correct")
else:
    print("Cluster 1 has the highest number of accidents with 'SAME DIRECTION': Incorrect")

# 9. Check if accidents in Cluster 1 are most frequent on Fridays compared to other days
friday_percentage = X[X['kmeans_cluster'] == 1]['Day_Friday'].mean() * 100
if friday_percentage > X[X['kmeans_cluster'] == 1][['Day_Monday', 'Day_Tuesday', 'Day_Wednesday', 'Day_Thursday', 'Day_Saturday', 'Day_Sunday']].mean(axis=1).max():
    print("Accidents in Cluster 1 are most frequent on Fridays: Correct")
else:
    print("Accidents in Cluster 1 are most frequent on Fridays: Incorrect")

# 10. Check if the number of accidents in Cluster 1 during “Morning Rush Hours” is higher than during “Evening Rush Hours”
morning_rush_percentage = X[X['kmeans_cluster'] == 1]['Time of Day_Morning Rush Hour'].mean() * 100
evening_rush_percentage = X[X['kmeans_cluster'] == 1]['Time of Day_Evening Rush Hour'].mean() * 100
if morning_rush_percentage > evening_rush_percentage:
    print("Number of accidents in Cluster 1 during 'Morning Rush Hours' is higher than during 'Evening Rush Hours': Correct")
else:
    print("Number of accidents in Cluster 1 during 'Morning Rush Hours' is higher than during 'Evening Rush Hours': Incorrect")


Cluster 1 has the lowest average 'Adjusted Average Daily Traffic Amount': Incorrect
Cluster 1 has the lowest average 'Speed Limit': Incorrect
The majority of the accidents in Cluster 1 occurred on roads classified as 'CITY STREET': Incorrect
The majority of accidents in Cluster 1 occurred on roads classified 'US & STATE HIGHWAYS': Correct
Cluster 1 is the only cluster with accidents recorded on 'NON TRAFFICWAY': Incorrect
Cluster 1 has the highest number of accidents on roadways with '4 OR MORE LANES, UNDIVIDED': Correct
Cluster 1 has the lowest number of accidents with 'ONE MOTOR VEHICLE': Correct
Cluster 1 has the highest number of accidents with 'SAME DIRECTION': Incorrect
Accidents in Cluster 1 are most frequent on Fridays: Correct
Number of accidents in Cluster 1 during 'Morning Rush Hours' is higher than during 'Evening Rush Hours': Incorrect


In [35]:
#15

In [36]:
# 1. Check if Cluster 2 has the highest average “Adjusted Average Daily Traffic Amount”
highest_traffic_amount_cluster = average_traffic_amount_kmeans.idxmax()
if highest_traffic_amount_cluster == 2:
    print("Cluster 2 has the highest average 'Adjusted Average Daily Traffic Amount': Correct")
else:
    print("Cluster 2 has the highest average 'Adjusted Average Daily Traffic Amount': Incorrect")

# 2. Check if Cluster 2 has the highest average “Speed Limit” compared to other clusters
highest_speed_limit_cluster = average_speed_limit_kmeans.idxmax()
if highest_speed_limit_cluster == 2:
    print("Cluster 2 has the highest average 'Speed Limit': Correct")
else:
    print("Cluster 2 has the highest average 'Speed Limit': Incorrect")

# 3. Check if the majority of the accidents in Cluster 2 occurred on roads classified as “CITY STREET”
city_street_percentage = X[X['kmeans_cluster'] == 2]['Road_CITY STREET'].mean() * 100
if city_street_percentage > 50:
    print("The majority of the accidents in Cluster 2 occurred on roads classified as 'CITY STREET': Correct")
else:
    print("The majority of the accidents in Cluster 2 occurred on roads classified as 'CITY STREET': Incorrect")

# 4. Check if none of the accidents in Cluster 2 occurred on “US & STATE HIGHWAYS”
us_state_highways_percentage = X[X['kmeans_cluster'] == 2]['Road_US & STATE HIGHWAYS'].mean() * 100
if us_state_highways_percentage == 0:
    print("None of the accidents in Cluster 2 occurred on 'US & STATE HIGHWAYS': Correct")
else:
    print("None of the accidents in Cluster 2 occurred on 'US & STATE HIGHWAYS': Incorrect")

# 5. Check if Cluster 2 is the only cluster with accidents recorded on “NON TRAFFICWAY” road class
non_trafficway_cluster_count = (X[X['Road_NON TRAFFICWAY'] == True]['kmeans_cluster'].nunique() == 1)
if non_trafficway_cluster_count and X[X['Road_NON TRAFFICWAY'] == True]['kmeans_cluster'].iloc[0] == 2:
    print("Cluster 2 is the only cluster with accidents recorded on 'NON TRAFFICWAY': Correct")
else:
    print("Cluster 2 is the only cluster with accidents recorded on 'NON TRAFFICWAY': Incorrect")

# 6. Check if Cluster 2 has the highest number of recorded accidents on roadways with “4 OR MORE LANES, DIVIDED”
highest_4_lanes_divided_cluster = X.groupby('kmeans_cluster')['Roadway_4 OR MORE LANES, DIVIDED'].sum().idxmax()
if highest_4_lanes_divided_cluster == 2:
    print("Cluster 2 has the highest number of accidents on roadways with '4 OR MORE LANES, DIVIDED': Correct")
else:
    print("Cluster 2 has the highest number of accidents on roadways with '4 OR MORE LANES, DIVIDED': Incorrect")

# 7. Check if Cluster 2 has the lowest number of recorded accidents with the collision type “ONE MOTOR VEHICLE”
lowest_one_motor_vehicle_cluster = collision_means['Collision manner_ONE MOTOR VEHICLE'].idxmin()
if lowest_one_motor_vehicle_cluster == 2:
    print("Cluster 2 has the lowest number of accidents with 'ONE MOTOR VEHICLE': Correct")
else:
    print("Cluster 2 has the lowest number of accidents with 'ONE MOTOR VEHICLE': Incorrect")

# 8. Check if Cluster 2 has the highest number of recorded accidents with the collision type “SAME DIRECTION”
highest_same_direction_cluster = collision_means['Collision manner_SAME DIRECTION'].idxmax()
if highest_same_direction_cluster == 2:
    print("Cluster 2 has the highest number of accidents with 'SAME DIRECTION': Correct")
else:
    print("Cluster 2 has the highest number of accidents with 'SAME DIRECTION': Incorrect")

# 9. Check if accidents in Cluster 2 are most frequent on Fridays compared to other days
friday_percentage = X[X['kmeans_cluster'] == 2]['Day_Friday'].mean() * 100
if friday_percentage > X[X['kmeans_cluster'] == 2][['Day_Monday', 'Day_Tuesday', 'Day_Wednesday', 'Day_Thursday', 'Day_Saturday', 'Day_Sunday']].mean(axis=1).max():
    print("Accidents in Cluster 2 are most frequent on Fridays: Correct")
else:
    print("Accidents in Cluster 2 are most frequent on Fridays: Incorrect")

# 10. Check if the number of accidents in Cluster 2 during “Morning Rush Hours” is higher than during “Evening Rush Hours”
morning_rush_percentage = X[X['kmeans_cluster'] == 2]['Time of Day_Morning Rush Hour'].mean() * 100
evening_rush_percentage = X[X['kmeans_cluster'] == 2]['Time of Day_Evening Rush Hour'].mean() * 100
if morning_rush_percentage > evening_rush_percentage:
    print("Number of accidents in Cluster 2 during 'Morning Rush Hours' is higher than during 'Evening Rush Hours': Correct")
else:
    print("Number of accidents in Cluster 2 during 'Morning Rush Hours' is higher than during 'Evening Rush Hours': Incorrect")

# 11. Check if none of the other choices are correct
if (highest_traffic_amount_cluster != 2 and
    highest_speed_limit_cluster != 2 and
    city_street_percentage <= 50 and
    us_state_highways_percentage != 0 and
    not non_trafficway_cluster_count and
    highest_4_lanes_divided_cluster != 2 and
    lowest_one_motor_vehicle_cluster != 2 and
    highest_same_direction_cluster != 2 and
    friday_percentage <= X[X['kmeans_cluster'] == 2][['Day_Monday', 'Day_Tuesday', 'Day_Wednesday', 'Day_Thursday', 'Day_Saturday', 'Day_Sunday']].mean(axis=1).max() and
    morning_rush_percentage <= evening_rush_percentage):
    print("None of the other choices are correct: Correct")
else:
    print("None of the other choices are correct: Incorrect")


Cluster 2 has the highest average 'Adjusted Average Daily Traffic Amount': Correct
Cluster 2 has the highest average 'Speed Limit': Correct
The majority of the accidents in Cluster 2 occurred on roads classified as 'CITY STREET': Incorrect
None of the accidents in Cluster 2 occurred on 'US & STATE HIGHWAYS': Correct
Cluster 2 is the only cluster with accidents recorded on 'NON TRAFFICWAY': Incorrect
Cluster 2 has the highest number of accidents on roadways with '4 OR MORE LANES, DIVIDED': Correct
Cluster 2 has the lowest number of accidents with 'ONE MOTOR VEHICLE': Incorrect
Cluster 2 has the highest number of accidents with 'SAME DIRECTION': Correct
Accidents in Cluster 2 are most frequent on Fridays: Correct
Number of accidents in Cluster 2 during 'Morning Rush Hours' is higher than during 'Evening Rush Hours': Incorrect
None of the other choices are correct: Incorrect


In [37]:
#16

In [38]:
# Step 1: Filter the data where 'Crash Severity' is 'SUSPECTED SERIOUS INJURY'
serious_injury_data = X[X['Crash Severity'] == 4]

# Step 2: Group by 'kmeans_cluster' and count the number of instances in each cluster
serious_injury_counts = serious_injury_data.groupby('kmeans_cluster').size()

# Step 3: Sort the clusters in ascending order based on the counts
sorted_clusters = serious_injury_counts.sort_values(ascending=True)

# Display the result
print("Clusters arranged in ascending order based on the number of 'SUSPECTED SERIOUS INJURY' instances:")
print(sorted_clusters)


Clusters arranged in ascending order based on the number of 'SUSPECTED SERIOUS INJURY' instances:
kmeans_cluster
2    14
0    25
1    33
dtype: int64


In [39]:
#17

In [40]:
# Step 1: Filter the data where 'Crash Severity' is 'FATAL'
fatal_accidents_data = X[X['Crash Severity'] == 5]

# Step 2: Group by 'kmeans_cluster' and count the number of instances in each cluster
fatal_accidents_counts = fatal_accidents_data.groupby('kmeans_cluster').size()

# Step 3: Check if Cluster 0 has the lowest number of fatal accidents
lowest_fatal_cluster = fatal_accidents_counts.idxmin()

# Output the result
if lowest_fatal_cluster == 0:
    print("True - Cluster 0 has the lowest number of fatal accidents.")
else:
    print("False - Cluster 0 does not have the lowest number of fatal accidents.")


True - Cluster 0 has the lowest number of fatal accidents.


In [41]:
#18

In [42]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Initialize a dictionary to store silhouette scores for different cluster numbers
silhouette_scores = {}

# Loop over different numbers of clusters (from 2 to 9)
for n_clusters in range(2, 10):
    # Step 1: Initialize the KMeans model with n_clusters and other specified parameters
    kmeans_model = KMeans(n_clusters=n_clusters, random_state=0, n_init='auto')
    
    # Step 2: Fit the model on the scaled dataset
    kmeans_model.fit(X_scaled)
    
    # Step 3: Predict cluster labels for the dataset
    cluster_labels = kmeans_model.labels_
    
    # Step 4: Compute the silhouette score
    score = silhouette_score(X_scaled, cluster_labels)
    
    # Step 5: Store the silhouette score for this number of clusters
    silhouette_scores[n_clusters] = score

# Find the number of clusters with the highest silhouette score
best_n_clusters = max(silhouette_scores, key=silhouette_scores.get)

# Output the silhouette scores and the best number of clusters
print("Silhouette scores for different numbers of clusters:")
for n_clusters, score in silhouette_scores.items():
    print(f"{n_clusters} clusters: {score:.4f}")

print(f"\nThe number of clusters with the highest silhouette score is: {best_n_clusters}")

Silhouette scores for different numbers of clusters:
2 clusters: 0.1710
3 clusters: 0.1739
4 clusters: 0.1599
5 clusters: 0.1457
6 clusters: 0.1633
7 clusters: 0.1519
8 clusters: 0.1569
9 clusters: 0.1508

The number of clusters with the highest silhouette score is: 3


In [43]:
#19

In [44]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Initialize a dictionary to store silhouette scores for different cluster numbers
silhouette_scores = {}

# Loop over different numbers of clusters (from 2 to 9)
for n_clusters in range(2, 10):
    # Step 1: Initialize the KMeans model with n_clusters and other specified parameters
    kmeans_model = KMeans(n_clusters=n_clusters, random_state=0, n_init='auto')
    
    # Step 2: Fit the model on the scaled dataset
    kmeans_model.fit(X_scaled)
    
    # Step 3: Predict cluster labels for the dataset
    cluster_labels = kmeans_model.labels_
    
    # Step 4: Compute the silhouette score
    score = silhouette_score(X_scaled, cluster_labels)
    
    # Step 5: Store the silhouette score for this number of clusters
    silhouette_scores[n_clusters] = score

# Find the number of clusters with the highest silhouette score
best_n_clusters = max(silhouette_scores, key=silhouette_scores.get)

# Fetch the silhouette score for the best number of clusters
best_silhouette_score = silhouette_scores[best_n_clusters]

# Print the silhouette score rounded to two decimal places
print(f"The silhouette score for {best_n_clusters} clusters is: {best_silhouette_score:.2f}")

The silhouette score for 3 clusters is: 0.17


In [45]:
#20

In [52]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering
import pandas as pd

# Define the range of clusters to evaluate and initialize a list for storing silhouette scores
cluster_options = list(range(2, 10))
silhouette_avg_scores = []

# Iterate over each cluster option to compute silhouette scores
for num_clusters in cluster_options:
    # Create an instance of AgglomerativeClustering with the current number of clusters
    clustering_model = AgglomerativeClustering(distance_threshold=None, n_clusters=num_clusters)
    # Fit the model and predict cluster labels for the data
    labels = clustering_model.fit_predict(X)
    # Calculate the silhouette score for the predicted labels and append it to the list
    silhouette_avg_scores.append(silhouette_score(X, labels))

# Create a DataFrame to store the number of clusters and their corresponding silhouette scores
silhouette_score_df = pd.DataFrame({
    'Num of Clusters': cluster_options,
    'Silhouette Avg Score': silhouette_avg_scores
})

# Set the number of clusters as the index of the DataFrame
silhouette_score_df.set_index('Num of Clusters', inplace=True)

# Print the number of clusters that achieved the highest silhouette score
print(silhouette_score_df['Silhouette Avg Score'].idxmax())



9


In [47]:
#21

In [51]:
for num_clusters in range(2, 10):
    clustering_model = AgglomerativeClustering(distance_threshold=None, n_clusters=num_clusters)
    labels = clustering_model.fit_predict(X)
    silhouette_avg = silhouette_score(X, labels)
    print(f"{num_clusters} Clusters: {round(silhouette_avg, 2)}")


2 Clusters: 0.81
3 Clusters: 0.76
4 Clusters: 0.7
5 Clusters: 0.74
6 Clusters: 0.79
7 Clusters: 0.82
8 Clusters: 0.84
9 Clusters: 0.87
